# Ejercicio 2 – Predicción de series temporales con ARIMA

## ¿Qué es ARIMA?

**ARIMA** (AutoRegressive Integrated Moving Average) es uno de los modelos estadísticos más utilizados para predecir series temporales. Funciona especialmente bien cuando se tiene una única variable a lo largo del tiempo y se busca capturar patrones como tendencias o ciclos.

Su potencia radica en tres componentes principales:

- **AR (Auto-Regresión)**: La serie se predice a partir de sus propios valores pasados.
- **I (Integración)**: Diferenciación para hacer la serie estacionaria.
- **MA (Media Móvil)**: Utiliza errores pasados para mejorar la predicción.

El modelo se representa como:  
📌 `ARIMA(p, d, q)`  
donde:
- `p`: representa el número de observaciones rezagadas (lags). Nos indica cuántos pasos atrás en el tiempo debemos mirar para predecir el valor actual. Se determina observando la autocorrelación de la serie.
- `d`: es el orden de diferenciación, es decir, cuántas veces hay que restar el valor actual con respecto al anterior para hacer que la serie sea estacionaria (sin tendencia ni variación en la varianza).
- `q`:  Representa cuántos errores de predicción pasados se incorporan en el modelo. A diferencia de una media móvil tradicional que suaviza datos, aquí se usa para modelar la relación entre los errores pasados y los valores actuales.

---

## Conceptos clave

- **Estacionariedad**: Una serie estacionaria mantiene sus propiedades estadísticas constantes a lo largo del tiempo. ARIMA requiere que la serie sea estacionaria para funcionar correctamente.
- **Autocorrelación**: Se refiere a cómo se relaciona una observación con valores pasados. Se mide con ACF y PACF.
- **Diferenciación**: Técnica para eliminar tendencia y hacer la serie estacionaria restando el valor anterior:  
  \[
  Y'_t = Y_t - Y_{t-1}
  \]

---

## Flujo de trabajo

```mermaid
flowchart TD
    A[Serie temporal entrenamiento] --> B[Test de estacionariedad ADF]
    B --> C[Aplicar diferenciación si es necesario]
    C --> D[Análisis ACF y PACF]
    D --> E[Selección de parámetros p, d, q]
    E --> F[Entrenamiento del modelo ARIMA]
    F --> G[Predicción y evaluación con set de validación]
    G --> H[Visualización y análisis de errores]
```

---

## Ejercicio práctico

### Paso 1: Cargar datos preparados (train / validation)

In [ ]:
import pandas as pd

# Cargar datasets desde archivo CSVs
train = pd.read_csv("data/m5_consumo_energia_train.csv", index_col=0)
val = pd.read_csv("data/m5_consumo_energia_test.csv", index_col=0)

# Establecer columna temporal como índice
train.set_index(pd.to_datetime(train.index), inplace=True)
# Cargar dataset desde archivo CSV

# Establecer columna temporal como índice
val.set_index(pd.to_datetime(val.index), inplace=True)

---

### Paso 2: Verificar estacionariedad con prueba ADF

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(train["Energia Consumida"].dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

if result[1] > 0.05:
    print("La serie NO es estacionaria. Se recomienda aplicar diferenciación.")
else:
    print("La serie es estacionaria.")

---

### Paso 3: Aplicar diferenciación si es necesario

In [ ]:
train_diff = train["Energia Consumida"].diff().dropna()

# Visualizar la serie diferenciada
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(train_diff)
plt.plot(train["Energia Consumida"])
plt.title("Serie diferenciada (1ª orden)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
result_diff = adfuller(train_diff)
print(f"ADF Statistic (Differenced): {result_diff[0]:.4f}")
print(f"p-value (Differenced): {result_diff[1]:.4f}")
if result_diff[1] < 0.05:
    print("La serie es estacionaria.")
else:
    print("La serie NO es estacionaria.")

---

### Paso 4: Estimar parámetros con ACF y PACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2, 1, figsize=(10, 6))
plot_acf(train_diff, ax=ax[0], lags=30)
plot_pacf(train_diff, ax=ax[1], lags=30)
plt.tight_layout()
plt.show()

> Con estos gráficos se seleccionan visualmente los valores `p` y `q`.

---

### Paso 5: Ajustar el modelo ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Definir y entrenar el modelo (por ejemplo ARIMA(p,d,q))
model = ARIMA(train["Energia Consumida"], order=(7, 0, 1))
model_fit = model.fit()

# Resumen del modelo
print(model_fit.summary())

---

### Paso 6: Hacer predicciones sobre el conjunto de validación

In [ ]:
# Predecir sobre el rango de fechas de validación
start = val.index[0]
end = val.index[-1]
pred = model_fit.predict(start=start, end=end, typ="levels")
# Intervalo de cnofianza
forecast = model_fit.get_forecast(steps=len(val))
pred_mean = forecast.predicted_mean
conf_int = forecast.conf_int() 
# Visualización
plt.figure(figsize=(12, 5))
plt.plot(train.index, train["Energia Consumida"], label="Train")
plt.plot(val.index, val["Energia Consumida"], label="Validation")
plt.plot(pred.index, pred, label="Predicción ARIMA", linestyle="--")
plt.fill_between(val.index,
                 conf_int.iloc[:, 0],  # límite inferior
                 conf_int.iloc[:, 1],  # límite superior
                 color='lightgreen', alpha=0.3, label="95% IC")
plt.title("Predicción de consumo con ARIMA")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

---

### Paso 7: Evaluar el rendimiento del modelo

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(val["Energia Consumida"], pred)
rmse = np.sqrt(mean_squared_error(val["Energia Consumida"], pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

### ¿Qué indican el MAE y el RMSE?

#### **MAE – Mean Absolute Error (Error Absoluto Medio)**

- **Qué mide:**  
  El promedio de los errores absolutos entre los valores reales y los predichos.

- **Fórmula:**  
  \[
  \text{MAE} = \frac{1}{n} \sum_{i=1}^{n} \left| y_i - \hat{y}_i \right|
  \]

- **Interpretación:**  
  Nos dice, en promedio, cuánto se desvían nuestras predicciones de los valores reales.  
  Por ejemplo, un MAE de 4 significa que, en promedio, nos estamos equivocando en ±4 unidades (p. ej., kWh, litros, etc.).

- **Ventaja:**  
  Fácil de interpretar, no amplifica los errores grandes.

---

#### **RMSE – Root Mean Squared Error (Raíz del Error Cuadrático Medio)**

- **Qué mide:**  
  El promedio del cuadrado de los errores, pero toma la **raíz cuadrada** para mantener las unidades originales.

- **Fórmula:**  
  \[
  \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
  \]

- **Interpretación:**  
  Similar al MAE, pero **penaliza más los errores grandes**. Es más sensible a valores atípicos (outliers).

---

## ✅ Resultado esperado

- Verificar la estacionariedad de una serie temporal.
- Aplicar diferenciación y análisis ACF/PACF para determinar parámetros.
- Ajustar un modelo ARIMA con `statsmodels`.
- Evaluar su rendimiento y analizar visualmente los resultados.
